In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium, fiona, branca

In [ ]:
import shapefile

In [ ]:
myshpfile = gpd.read_file('ecuador.shp')
myshpfile.to_file('map.geojson', driver='GeoJSON')

In [2]:
provincias=gpd.read_file("map.geojson")

In [ ]:
provincias

In [3]:
regiones = pd.read_csv("../proyecto/regiones.csv")

In [20]:
region1=regiones[["address_region","cluster_1"]]

In [21]:
region1.columns=["DPA_DESPRO",'cluster_1']

In [22]:
rg1=provincias.merge(region1,on="DPA_DESPRO",how="left")

In [4]:
region0=regiones.iloc[:,:2]

In [5]:
region0.columns=["DPA_DESPRO",'cluster_0']

In [6]:
rg0=provincias.merge(region0,on="DPA_DESPRO",how="left")

In [7]:
rg0

,DPA_DESPRO,geometry,cluster_0
0,SANTA ELENA,"POLYGON ((-81.00769 -2.18026, -80.56824 -2.515...",453.0
1,MANABI,"POLYGON ((-80.84290 -1.56536, -80.85937 -1.587...",481.0
2,ESMERALDAS,"POLYGON ((-79.76074 0.36804, -79.70581 0.15930...",172.0
3,GUAYAS,"POLYGON ((-80.55176 -2.52604, -80.40894 -2.657...",823.0
4,LOS RIOS,"POLYGON ((-79.51904 -1.07110, -79.75525 -1.257...",118.0
5,EL ORO,"POLYGON ((-79.85413 -3.04178, -79.98596 -3.403...",263.0
6,CARCHI,"POLYGON ((-78.50830 1.19193, -78.54675 1.00520...",282.0
7,IMBABURA,"POLYGON ((-78.44238 0.60424, -78.65662 0.36255...",173.0
8,PICHINCHA,"POLYGON ((-79.34326 0.03296, -79.03564 -0.0329...",1446.0
9,SANTO DOMINGO DE LOS TSACHILAS,"POLYGON ((-79.62891 -0.01648, -79.54651 -0.197...",79.0


In [8]:
rg0.to_file("output.geojson",driver="GeoJSON")

In [23]:
rg1.to_file("outpu1.geojson",driver="GeoJSON")

In [ ]:
rg0=gpd.read_file("output.geojson")

In [ ]:
rg0["geometry"]=rg0["geometry"].to_crs(4326)

In [24]:
centroid=rg1.geometry.centroid
mapbox_api_key = r"pk.eyJ1Ijoic3RhbWVuIiwiYSI6IlpkZEtuS1EifQ.jiH_c9ShtBwtqH9RdG40mw"; tileset_ID_str = "streets-v11"; tilesize_pixels = "256";
m=folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=12,
             tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}",
             attr='MAPBOX')

c:\users\sof-d\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [25]:
field = 'cluster_1'
label = "Entidades pertenecientes al cluster 0"
colormap = branca.colormap.linear.YlOrBr_04.to_step(4).scale(rg1[field].min(),rg1[field].max())
colormap.caption = label
layer = folium.GeoJson('outpu1.geojson',
        name=label,
        style_function=lambda x: {"weight":2, 'color':'black','fillColor':colormap(x['properties'][field]), 'fillOpacity':1},
        highlight_function=lambda x: {'weight':5, 'color':'black'},
        smooth_factor=2.0,
        tooltip=folium.features.GeoJsonTooltip(fields=['DPA_DESPRO',field],
                                      aliases=['DPA_DESPRO',label], 
                                      labels=True, 
                                      sticky=True,
                                      localize=True
                                     ),
        show = True,
        overlay = True
        )
m.add_child(layer)
m.add_child(colormap)